Textbook exercises

Answer the following questions:

16.3-2 Prove that a binary tree that is not full cannot correspond to an optimal prefix code.

16.3-3 What is an optimal Huffman code for the following set of frequencies, based on the first 8 Fibonacci numbers? {a:1 b:1 c:2 d:3 e:5 f:8 g:13 h:21} Can you generalize your answer to find the optimal code when the frequencies are the first n Fibonacci numbers?

16.3-8 [Optional] Suppose that a data file contains a sequence of 8-bit characters such that all 256 characters are about equally common: the maximum character frequency is less than twice the minimum character frequency. Prove that Huffman coding in this case is no more efficient than using an ordinary 8-bit fixed-length code.

​

Huffman encoding

There is python code available at this gist, which will download the collected works of Shakespeare and, when complete, it will compress them using the Huffman compression algorithm.

Implement encode so that it takes a dictionary which maps symbols to probabilities, and then produces the optimal Huffman encoding (i.e. it must return a new dictionary).
Run the code and compress the complete works of Shakespeare. How big is the original file, what is the size of the compressed version? When the compressed version is decompressed is it identical to the original?
Compress the complete works of Shakespeare using other compression methods and see what a poor job they do (eg. gzip, bzip2, zip). Bear in mind that we are using the optimal symbol code with frequencies constructed from the exact corpus that we are looking to compress, so nothing can beat us right?
What is the percentage of 1’s in the compressed version and what is the percentage of 1’s in the uncompressed version?
Create a Google document to record all your answers and include there a link to a gist you have created with your code (here are two videos, video1 and video2, that illustrate how you can convert your python script and CoCalc/Jupyter Notebook into a gist). Be ready to paste a link to your Google document into a class poll (make sure the document’s settings are such that one can assess its content from the link).

